In [6]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
import random
%matplotlib inline

In [7]:
# read in all the words
words = open('data/names.txt', 'r').read().splitlines()
print(len(words))
print(max(len(w) for w in words))
print(words[:8])
# build the vocabulary of characters and mappings to/from integers


chars = sorted(list(set(''.join(words))))
stoi = {s: i + 1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i: s for s, i in stoi.items()}
vocab_size = len(itos)
print(f'Character to index mapping: {itos}')
print(f'Vocabulary size: {vocab_size}')

# Shuffle the words
random.seed(42)
random.shuffle(words)


32033
15
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']
Character to index mapping: {1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
Vocabulary size: 27


In [93]:
import torch

def encode_words(words):
    encoded = []
    for w in words:
        encoded.extend([stoi[ch] for ch in '.' + w + '.'])
    return encoded

encoded = encode_words(words)

def create_pairs(seq, block_size):
    X, Y = [], []
    for i in range(0, len(seq) - block_size):
        X.append(seq[i:i+block_size])
        Y.append(seq[i+1:i+block_size+1])
    X = torch.tensor(X, dtype=torch.long)
    Y = torch.tensor(Y, dtype=torch.long)
    return X, Y

n = len(encoded)
n1 = int(0.8 * n)

train_seq = encoded[:n1]
dev_seq = encoded[n1:]

block_size = 8

Xtr, Ytr = create_pairs(train_seq, block_size)
Xdev, Ydev = create_pairs(dev_seq, block_size)

print(f'Training data shapes - X: {Xtr.shape}, Y: {Ytr.shape}')
print(f'Development data shapes - X: {Xdev.shape}, Y: {Ydev.shape}')

def split_into_batches(X, Y, batch_size):
    num_batches = X.size(0) // batch_size
    X = X[:num_batches * batch_size]
    Y = Y[:num_batches * batch_size]
    
    X = X.view(batch_size, -1, X.size(-1))
    Y = Y.view(batch_size, -1, Y.size(-1))
    return X, Y

Xtr_batched, Ytr_batched = split_into_batches(Xtr, Ytr, batch_size)
Xdev_batched, Ydev_batched = split_into_batches(Xdev, Ydev, batch_size)

print(f'Training batches shape - X: {Xtr_batched.shape}, Y: {Ytr_batched.shape}')
print(f'Development batches shape - X: {Xdev_batched.shape}, Y: {Ydev_batched.shape}')


Training data shapes - X: torch.Size([208135, 8]), Y: torch.Size([208135, 8])
Development data shapes - X: torch.Size([52028, 8]), Y: torch.Size([52028, 8])
Training batches shape - X: torch.Size([32, 6504, 8]), Y: torch.Size([32, 6504, 8])
Development batches shape - X: torch.Size([32, 1625, 8]), Y: torch.Size([32, 1625, 8])


In [135]:
one_hot_Xtr = F.one_hot(Xtr, 27)
one_hot_Xtr = one_hot_Xtr.view(8, -1, 27)
one_hot_Xtr.shape

torch.Size([8, 208135, 27])

In [136]:
batch_size = 32
hidden_size = 30
# Get a batch of random numbers
batch = torch.randint(0, one_hot_Xtr.shape[1], (batch_size,))
Xb = one_hot_Xtr[:, batch, :] # (8, 32, 27)
Yb = Ytr[batch]
Yb = Yb.view(8, -1)
time_steps, batch_size, input_size = Xb.shape
print(time_steps)
print(batch_size)
print(input_size)
Xb.shape, Yb.shape

8
32
27


(torch.Size([8, 32, 27]), torch.Size([8, 32]))

In [137]:
Xb = Xb.type(torch.float32)
Yb = Yb.type(torch.int64)

# Parameters
Fvh = torch.randn(vocab_size, hidden_size)
i1vh = torch.randn(vocab_size, hidden_size)
i2vh = torch.randn(vocab_size, hidden_size)
Ovh = torch.randn(vocab_size, hidden_size)

Fhh = torch.randn(hidden_size, hidden_size)
i1hh = torch.randn(hidden_size, hidden_size)
i2hh = torch.randn(hidden_size, hidden_size)
Ohh = torch.randn(hidden_size, hidden_size)

bias1 = torch.zeros(hidden_size)
bias2 = torch.zeros(hidden_size)
bias3 = torch.zeros(hidden_size)
bias4 = torch.zeros(hidden_size)

output_matrix = torch.randn(hidden_size, vocab_size)

# Storage
preact1 = torch.zeros(time_steps, batch_size, hidden_size)
preact2 = torch.zeros(time_steps, batch_size, hidden_size)
preact3 = torch.zeros(time_steps, batch_size, hidden_size)
preact4 = torch.zeros(time_steps, batch_size, hidden_size)

act1 = torch.zeros(time_steps, batch_size, hidden_size)
act2 = torch.zeros(time_steps, batch_size, hidden_size)
act3 = torch.zeros(time_steps, batch_size, hidden_size)
act4 = torch.zeros((time_steps, batch_size, hidden_size))

Cin = torch.zeros((time_steps, batch_size, hidden_size))
Cout = torch.zeros((time_steps, batch_size, hidden_size))
Ctout = torch.zeros((time_steps, batch_size, hidden_size))

Hin = torch.zeros((time_steps, batch_size, hidden_size))
Hout = torch.zeros((time_steps, batch_size, hidden_size))

logits = torch.zeros((time_steps, batch_size, vocab_size))

c0 = torch.zeros(batch_size, hidden_size)
h0 = torch.zeros((batch_size, hidden_size))

# Backward pass

# To update
dFvh = torch.zeros(vocab_size, hidden_size) / 0.0
di1vh = torch.zeros(vocab_size, hidden_size) / 0.0
di2vh = torch.zeros(vocab_size, hidden_size) / 0.0
dOvh = torch.zeros(vocab_size, hidden_size) / 0.0

dFhh = torch.zeros(hidden_size, hidden_size)/ 0.0
di1hh = torch.zeros(hidden_size, hidden_size)/ 0.0
di2hh = torch.zeros(hidden_size, hidden_size)/ 0.0
dOhh = torch.zeros(hidden_size, hidden_size)/ 0.0

dbias1 = torch.zeros(hidden_size) # (30)
dbias2 = torch.zeros(hidden_size) # (30)
dbias3 = torch.zeros(hidden_size) # (30)
dbias4 = torch.zeros(hidden_size) # (30)

doutput_matrix = torch.randn(hidden_size, vocab_size) # (30, 27)


# Placeholders (indexed with t)
dlogits = torch.zeros((time_steps, batch_size, vocab_size)) # (30, 27)
dhidden1 = torch.zeros(time_steps, batch_size, hidden_size) # (32, 30)
dhidden2 = torch.zeros(time_steps, batch_size, hidden_size) # (32, 30)
dtotal = torch.zeros(time_steps, batch_size, hidden_size) # (32, 30)

dpreact1 = torch.zeros(time_steps, batch_size, hidden_size) # (32, 30) 
dpreact2 = torch.zeros(time_steps, batch_size, hidden_size) # (32, 30)
dpreact3 = torch.zeros(time_steps, batch_size, hidden_size) # (32, 30)
dpreact4 = torch.zeros(time_steps, batch_size, hidden_size) # (32, 30)

dact1 = torch.zeros(time_steps, batch_size, hidden_size) # (32, 30) 
dact2 = torch.zeros(time_steps, batch_size, hidden_size) # (32, 30)
dact3 = torch.zeros(time_steps, batch_size, hidden_size) # (32, 30)
dact4 = torch.zeros((time_steps, batch_size, hidden_size)) # (32, 30)

dC = torch.zeros((time_steps, batch_size, hidden_size)) # (32, 30)
dCt = torch.zeros((time_steps, batch_size, hidden_size)) # (32, 30)
dHin = torch.zeros((time_steps, batch_size, hidden_size)) # (32, 30)
dHout = torch.zeros((time_steps, batch_size, hidden_size)) # (32, 30)
dlogits = torch.zeros((time_steps, batch_size, vocab_size)) # (32, 27)

dc0 = torch.zeros(batch_size, hidden_size)
dh0 = torch.zeros((batch_size, hidden_size))
dcn = torch.zeros(batch_size, hidden_size)
dhn = torch.zeros((batch_size, hidden_size))

In [138]:
batch_size = 32
hidden_size = 30
lr = 0.1
time_steps = 8
batch_size = 32
nput_size = 27
Xb.shape, Yb.shape
iterations = 0
h0 = torch.zeros((batch_size, hidden_size))
c0 = torch.zeros((batch_size, hidden_size))

while iterations < 2000:
    # Get a batch of random numbers into correct shape
    batch = torch.randint(0, one_hot_Xtr.shape[1], (batch_size,))
    Xb = one_hot_Xtr[:, batch, :] # (8, 32, 27)
    Xb = Xb / 1.0
    Yb = Ytr[batch]
    Yb = Yb.view(8, -1)

    # Forward propagation
    for t in range(time_steps):
        # if c0 is None: c0 = np.zeros((b,d))
        # if h0 is None: h0 = np.zeros((b,d))
        if t == 0:
            Hin[t] = h0
            Cin[t] = c0
        else:
            Hin[t] = Hout[t-1]
            Cin[t] = Cout[t-1]
        loss = 0
        
        preact1[t] = Xb[t] @ Fvh + Hin[t] @ Fhh + bias1 # (32, 27) @ (27, 30) + (32, 30) @ (30, 30) + (30)
        preact2[t] = Xb[t] @ i1vh + Hin[t] @ i1hh * bias2 # (32, 27) @ (27, 30) + (32, 30) @ (30, 30) + (30)
        preact3[t] = Xb[t] @ i2vh + Hin[t] @ i2hh + bias3 # (32, 27) @ (27, 30) + (32, 30) @ (30, 30) + (30)
        preact4[t] = Xb[t] @ Ovh + Hin[t] @ Ohh + bias4 # (32, 27) @ (27, 30) + (32, 30) @ (30, 30) + (30)
        
        act1[t] = torch.sigmoid(preact1[t]) # (32, 30)
        act2[t] = torch.sigmoid(preact2[t]) # (32, 30)
        act3[t] = torch.tanh(preact3[t]) # (32, 30)
        act4[t] = torch.sigmoid(preact4[t]) # (32, 30)
        
        Cout[t] = act1[t] * Cin[t] + act2[t] * act3[t] # (32, 30)
        if t < time_steps -1: Cin[t+1] = Cout[t]
        Ctout[t] = torch.tanh(Cout[t]) # (32, 30)
        Hout[t] = Ct[t] * act4[t] # (32, 30)
        if t < time_steps -1: Hin[t+1] = Hout[t] 
        
        
        logits[t] = Hout[t] @ output_matrix # (32, 27)
        counts = logits.exp()
        counts_sum = counts.sum(1, keepdims=True)
        counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
        probs = counts * counts_sum_inv
        logprobs = probs.log()
        loss += -logprobs[t][torch.arange(32), Yb].mean()

    if (iterations % 100 == 0):
        print (loss / time_steps)
    
### ------------------------------------------------------------------------------------------------------------------
    # Backward pass
    for t in reversed(range(time_steps)):
    
        # Backpropogate cross entropy
        dlogits[t] = F.softmax(logits[t], 1)
        dlogits[t][torch.arange(batch_size), Yb] -= 1
        dlogits /= n
        
        # Backpropogate dHout
        # t = 7, 6, 5, 4, 3, 2, 1, 0
        if (t < time_steps-1):
            # dHout of a previous time step, must add dHin of the next time step
            # dHout of one time step, becomes the input for the next time step
            # Hout[t] = Hin[t+1]
            dHout[t] = dHin[t+1]
            dHout[t] =  dlogits[t] @ output_matrix.T + dHin[t+1] # (32, 27) @ (27, 30) = (32, 30) dHt on paper derivation
        else: # t = 7
            dHout[t] =  dlogits[t] @ output_matrix.T + dhn
    
        # Backpropogate output matrix
        doutput_matrix = dlogits[t].T @ dHout[t] # (32, 27)
        
        # Backpropogate dact3 (output gate activations)
        dact3[t] = dHout[t] * Ctout[t] # (32, 27) * (32, 27) = (32, 27)
        
        # Backpropogate dC (current cell state)
        dC[t] = dHout[t] * act4[t] * (1 - torch.tanh(Cout[t])**2) # (32, 27) * (32, 27) * (32, 27) = (32, 27)
        
        # Backpropogate act1 and previous cell state
        if t > 0:
            # Forget gate activations
            # Last cell activations
            dact1[t] = dC[t] * Cout[t-1] # (32, 27) * (32, 27) = (32, 27)
            dC[t-1] = dC[t] * act1[t]
        else:
            dact1[t] = dC[t] * c0
            dc0 = dC[t] * act1[t]
        
        # Backpropogate i1 activations
        dact2[t] = dC[t] * act3[t]
        
        # Backpropogate i2 activations
        dact3[t] = dC[t] * act2[t]
        
        # Backpropogate all preactivations
        dpreact1[t] = dact1[t] * act1[t] * ( 1- act1[t])
        dpreact2[t] = dact2[t] * act2[t] * ( 1- act2[t])
        dpreact3[t] = dact3[t] * (1 - torch.tanh(dpreact3[t])**2)
        dpreact4[t] = dact4[t] * act4[t] * ( 1- act4[t])
        
        # Backpropogate gates
        dFvh = Xb[t].T @ dpreact1[t] # (27, 32) (32, 30) = (27, 30)
        dFhh = Hin[t].T @ dpreact1[t] 
        di1vh = Xb[t].T @ dpreact2[t]
        di1hh = Hin[t].T @ dpreact2[t]
        di2vh = Xb[t].T @ dpreact3[t]
        di2hh = Hin[t].T @ dpreact3[t]
        dOvh = Xb[t].T @ dpreact4[t]
        dOhh = Hin[t].T @ dpreact4[t]
        
        # Backpropogate prevh
        dHin[t] = dpreact1[t] @ Fhh.T + dpreact2[t] @ i1hh.T + dpreact3[t] @ i2hh.T + dpreact4[t] @ Ohh.T

### ------------------------------------------------------------------------------------------------------------------

    # Update parameters using gradients
    Fvh -= lr * dFvh
    i1vh -= lr * di1vh
    i2vh -= lr * di2vh
    Ovh -= lr * dOvh
    
    Fhh -= lr * dFhh
    i1hh -= lr * di1hh
    i2hh -= lr * di2hh
    Ohh -= lr * dOhh
    
    bias1 -= lr * dbias1
    bias2 -= lr * dbias2
    bias3 -= lr * dbias3
    bias4 -= lr * dbias4
    h0 = Hout[-1]
    c0 = Cout[-1]

    iterations+= 1

tensor(0.5591)
tensor(0.5620)
tensor(0.5519)
tensor(0.5470)
tensor(0.5595)
tensor(0.5681)
tensor(0.5561)
tensor(0.5641)
tensor(0.5650)


KeyboardInterrupt: 

In [162]:
import torch

def sigmoid(x):
    return 1 / (1 + torch.exp(-x))

def softmax(x):
    e_x = torch.exp(x - torch.max(x, dim=1, keepdim=True)[0])
    return e_x / torch.sum(e_x, dim=1, keepdim=True)

def sample_model(start_vector, Fvh, Fhh, i1vh, i1hh, i2vh, i2hh, Ovh, Ohh, bias1, bias2, bias3, bias4, output_matrix):
    """
    Generate a sequence of samples from the model.
    
    Parameters:
    - start_vector: Initial one-hot encoded vector to start the generation (torch tensor of shape (1, input_size))
    - Other parameters: Model weights and biases
    
    Returns:
    - Generated sequence (list of integers)
    """
    input_size = start_vector.shape[1]
    hidden_size = Fvh.shape[1]
    
    # Initialize hidden and cell states
    h = torch.zeros((1, hidden_size))
    c = torch.zeros((1, hidden_size))
    
    # Initialize the generated sequence with the index of the start vector
    generated_sequence = [torch.argmax(start_vector).item()]
    
    # Loop until '.' is generated or the maximum sequence length is reached
    while generated_sequence[-1] != 0 and len(generated_sequence) <= max_length:
        x = start_vector  # Use the provided start vector as input
        
        preact1 = x @ Fvh + h @ Fhh + bias1
        preact2 = x @ i1vh + h @ i1hh + bias2
        preact3 = x @ i2vh + h @ i2hh + bias3
        preact4 = x @ Ovh + h @ Ohh + bias4
        
        act1 = torch.sigmoid(preact1)
        act2 = torch.sigmoid(preact2)
        act3 = torch.tanh(preact3)
        act4 = torch.sigmoid(preact4)
        
        c = act1 * c + act2 * act3
        h = torch.tanh(c) * act4
        
        logits = h @ output_matrix
        probs = softmax(logits)
        
        # Sample from the probability distribution to get the next input
        next_index = torch.multinomial(probs.squeeze(), 1).item()
        
        generated_sequence.append(next_index)
        
        # Update the start vector with the one-hot encoded representation of the next index
        start_vector = torch.zeros((1, input_size))
        start_vector[0, next_index] = 1
        
    return [itos[ch] for ch in generated_sequence]

# Example usage
input_size = 27  # Size of the one-hot encoded vector
start_index = 1  # Index representing the starting point (e.g., 'a')
start_vector = torch.zeros((1, input_size))
start_vector[0, start_index] = 1  # One-hot encode the starting point

# Assume Fvh, Fhh, i1vh, i1hh, i2vh, i2hh, Ovh, Ohh, bias1, bias2, bias3, bias4, and output_matrix are already defined

max_length = 100  # Maximum length of the generated sequence


aamjaomjmjfbffstsxmddzmjmsmejomjrpmldsszmddajkdzr.


In [172]:
generated_sequence = sample_model(start_vector, Fvh, Fhh, i1vh, i1hh, i2vh, i2hh, Ovh, Ohh, bias1, bias2, bias3, bias4, output_matrix)
print(''.join(generated_sequence))

alkbsetbrmrokmommssmrwqudkmjsnzjtomsjmdhdrijowwhh.


In [124]:
def softmax(x):
    e_x = torch.exp(x - torch.max(x, axis=1, keepdims=True))
    return e_x / torch.sum(e_x, axis=1, keepdims=True)

In [152]:
# Forward pass

loss = 0
for t in range(time_steps):
    if c0 is None: c0 = np.zeros((b,d))
    if h0 is None: h0 = np.zeros((b,d))
    Hin[0] = h0
    Cin[0] =  c0
    
    preact1[t] = Xb[t] @ Fvh + Hin[t] @ Fhh + bias1 # (32, 27) @ (27, 30) + (32, 30) @ (30, 30) + (30)
    preact2[t] = Xb[t] @ i1vh + Hin[t] @ i1hh * bias2 # (32, 27) @ (27, 30) + (32, 30) @ (30, 30) + (30)
    preact3[t] = Xb[t] @ i2vh + Hin[t] @ i2hh + bias3 # (32, 27) @ (27, 30) + (32, 30) @ (30, 30) + (30)
    preact4[t] = Xb[t] @ Ovh + Hin[t] @ Ohh + bias4 # (32, 27) @ (27, 30) + (32, 30) @ (30, 30) + (30)
    
    act1[t] = torch.sigmoid(preact1[t]) # (32, 30)
    act2[t] = torch.sigmoid(preact2[t]) # (32, 30)
    act3[t] = torch.tanh(preact3[t]) # (32, 30)
    act4[t] = torch.sigmoid(preact4[t]) # (32, 30)
    
    Cout[t] = act1[t] * Cin[t] + act2[t] * act3[t] # (32, 30)
    if t < time_steps -1: Cin[t+1] = Cout[t]
    Ctout[t] = torch.tanh(Cout[t]) # (32, 30)
    Hout[t] = Ct[t] * act4[t] # (32, 30)
    if t < time_steps -1: Hin[t+1] = Hout[t] 
    
    
    logits[t] = Hout[t] @ output_matrix # (32, 27)
    counts = logits.exp()
    counts_sum = counts.sum(1, keepdims=True)
    counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
    probs = counts * counts_sum_inv
    logprobs = probs.log()
    loss += -logprobs[t][torch.arange(32), Yb].mean()

print (loss / time_steps)

tensor(4.2775)


In [147]:
for t in reversed(range(time_steps)):
    
    # Backpropogate cross entropy
    dlogits[t] = F.softmax(logits[t], 1)
    dlogits[t][torch.arange(batch_size), Yb] -= 1
    dlogits /= n
    
    # Backpropogate dHout
    # t = 7, 6, 5, 4, 3, 2, 1, 0
    if (t < time_steps-1):
        # dHout of a previous time step, must add dHin of the next time step
        # dHout of one time step, becomes the input for the next time step
        # Hout[t] = Hin[t+1]
        dHout[t] = dHin[t+1]
        dHout[t] =  dlogits[t] @ output_matrix.T + dHin[t+1] # (32, 27) @ (27, 30) = (32, 30) dHt on paper derivation
    else: # t = 7
        dHout[t] =  dlogits[t] @ output_matrix.T + dhn

    # Backpropogate output matrix
    doutput_matrix = dlogits[t].T @ dHout[t] # (32, 27)
    
    # Backpropogate dact3 (output gate activations)
    dact3[t] = dHout[t] * Ctout[t] # (32, 27) * (32, 27) = (32, 27)
    
    # Backpropogate dC (current cell state)
    dC[t] = dHout[t] * act4[t] * (1 - torch.tanh(Cout[t])**2) # (32, 27) * (32, 27) * (32, 27) = (32, 27)
    
    # Backpropogate act1 and previous cell state
    if t > 0:
        # Forget gate activations
        # Last cell activations
        dact1[t] = dC[t] * Cout[t-1] # (32, 27) * (32, 27) = (32, 27)
        dC[t-1] = dC[t] * act1[t]
    else:
        dact1[t] = dC[t] * c0
        dc0 = dC[t] * act1[t]
    
    # Backpropogate i1 activations
    dact2[t] = dC[t] * act3[t]
    
    # Backpropogate i2 activations
    dact3[t] = dC[t] * act2[t]
    
    # Backpropogate all preactivations
    dpreact1[t] = dact1[t] * act1[t] * ( 1- act1[t])
    dpreact2[t] = dact2[t] * act2[t] * ( 1- act2[t])
    dpreact3[t] = dact3[t] * (1 - torch.tanh(dpreact3[t])**2)
    dpreact4[t] = dact4[t] * act4[t] * ( 1- act4[t])
    
    # Backpropogate gates
    dFvh = Xb[t].T @ dpreact1[t] # (27, 32) (32, 30) = (27, 30)
    dFhh = Hin[t].T @ dpreact1[t] 
    di1vh = Xb[t].T @ dpreact2[t]
    di1hh = Hin[t].T @ dpreact2[t]
    di2vh = Xb[t].T @ dpreact3[t]
    di2hh = Hin[t].T @ dpreact3[t]
    dOvh = Xb[t].T @ dpreact4[t]
    dOhh = Hin[t].T @ dpreact4[t]
    
    # Backpropogate prevh
    dHin[t] = dpreact1[t] @ Fhh.T + dpreact2[t] @ i1hh.T + dpreact3[t] @ i2hh.T + dpreact4[t] @ Ohh.T


In [148]:
# Learning rate
lr = 0.5

# Update parameters using gradients
Fvh -= lr * dFvh
i1vh -= lr * di1vh
i2vh -= lr * di2vh
Ovh -= lr * dOvh

Fhh -= lr * dFhh
i1hh -= lr * di1hh
i2hh -= lr * di2hh
Ohh -= lr * dOhh

bias1 -= lr * dbias1
bias2 -= lr * dbias2
bias3 -= lr * dbias3
bias4 -= lr * dbias4